In [1]:
# Se importan la libreria necesaria para el proyecto

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
import os

In [6]:
# Leemos el archivos cvs
df = pd.read_csv(./Users/fdi/Documents/bootcamp/modulo_01/proyecto_modulo_1/mcp_1.csv)

SyntaxError: invalid syntax (4232284872.py, line 2)